In [15]:
import akshare as ak
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler


# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

def get_cs_index_stocks():
    """
    获取中证指数成分股（如中证红利 000922）
    兼容 akshare 最新列名
    """
    print("正在获取指数成分股...")
    try:
        df = ak.index_stock_cons_csindex(symbol="932368")  # 中证红利指数
        print("原始列名:", df.columns.tolist())

        # 检查必要列是否存在
        if "成分券代码" not in df.columns or "成分券名称" not in df.columns:
            print("错误：缺少必要列 '成分券代码' 或 '成分券名称'")
            return pd.DataFrame(columns=["stock_code", "stock_name"])

        # 提取所需列并重命名
        df = df[["成分券代码", "成分券名称"]].copy()
        df.columns = ["stock_code", "stock_name"]

        # 去除 .SH / .SZ 后缀
        df["stock_code"] = df["stock_code"].str.replace(r"\.SH|\.SZ", "", regex=True)

        print(f"成功获取 {len(df)} 只成分股")
        return df

    except Exception as e:
        print("获取指数成分股失败:", e)
        return pd.DataFrame(columns=["stock_code", "stock_name"])

def get_spot_data_for_stocks(stocks_df):
    """
    一次性获取所有成分股的实时行情数据
    """
    print("正在获取所有成分股的实时行情数据...")
    
    # 获取所有股票的代码列表
    stock_codes = stocks_df["stock_code"].tolist()
    
    try:
        # 一次性获取所有股票的实时数据
        spot_df = ak.stock_zh_a_spot_em()
        
        # 筛选出我们需要的成分股
        spot_df = spot_df[spot_df["代码"].isin(stock_codes)].copy()
        
        if spot_df.empty:
            print("未找到成分股的实时数据")
            return pd.DataFrame()
        
        # 重命名列以便使用
        spot_df = spot_df.rename(columns={
            "代码": "code",
            "名称": "name",
            "市净率": "pb",
            "市盈率-动态": "pe_ttm",
            "总市值": "market_cap"
        })
        
        # 选择需要的列
        spot_df = spot_df[["code", "name", "pb", "pe_ttm", "market_cap"]].copy()
        
        # 记录原始数据量
        original_count = len(spot_df)
        print(f"获取到 {original_count} 只股票的实时数据")
        
        # 过滤异常值：剔除PB<=0、PE_TTM<=0或缺失的值
        spot_df = spot_df[spot_df["pb"] > 0]
        print(f"过滤PB<=0后剩余: {len(spot_df)} 只")
        
        spot_df = spot_df[spot_df["pe_ttm"] > 0]
        print(f"过滤PE_TTM<=0后剩余: {len(spot_df)} 只")
        
        spot_df = spot_df.dropna(subset=["pb", "pe_ttm", "market_cap"])
        print(f"去除缺失值后剩余: {len(spot_df)} 只")
        
        if spot_df.empty:
            print("没有有效的实时数据")
            return pd.DataFrame()
        
        print(f"成功获取 {len(spot_df)} 只股票的有效实时数据")
        return spot_df
        
    except Exception as e:
        print(f"获取实时数据失败: {e}")
        return pd.DataFrame()

def get_low_pb_stocks(spot_df, top_n=10):
    """
    从实时数据中筛选出 PB 最低的 top_n 只股票
    """
    print("正在筛选低PB股票...")
    
    if spot_df.empty:
        print("实时数据为空")
        return pd.DataFrame()
    
    # 确保数据已经过滤了PB<=0的情况
    valid_pb_df = spot_df[spot_df["pb"] > 0].copy()
    
    if valid_pb_df.empty:
        print("没有有效的PB数据")
        return pd.DataFrame()
    
    # 选择PB最小的top_n只股票
    result_df = valid_pb_df.sort_values("pb").head(top_n).reset_index(drop=True)
    
    return result_df[["code", "name", "pb", "market_cap"]]

def neutralize_factor_sklearn(target_factor, control_factors):
    """
    使用 sklearn LinearRegression 进行因子中性化
    参数:
        target_factor: 需要中性化的目标因子 (n_samples,)
        control_factors: 控制因子矩阵 (n_samples, n_features)
    返回:
        中性化后的因子 (残差)
    """
    if len(control_factors.shape) == 1:
        control_factors = control_factors.reshape(-1, 1)
    
    # 使用 LinearRegression 进行回归
    lr = LinearRegression(fit_intercept=True)
    lr.fit(control_factors, target_factor)
    
    # 计算预测值
    predicted = lr.predict(control_factors)
    
    # 返回残差（中性化后的因子）
    return target_factor - predicted

def standardize_wide(series):
    """
    横截面标准化函数
    """
    if series.std() == 0:
        return pd.Series(0, index=series.index)
    return (series - series.mean()) / series.std()

def get_low_pe_market_neutral_stocks(spot_df, top_n=10):
    """
    方案 A：使用统一的 neutralize_factor_sklearn 函数
    1. 对 log(PE_TTM) 做市值中性化（回归残差）。  
    2. 对残差与"小市值"分别做 Z-Score 标准化。  
    3. 等权打分：score = 0.5 * (-residual_z) + 0.5 * (-cap_z)  
       负号是因为残差越小、市值越小越好。  
    4. 取分值最小的 top_n 只股票。
    """
    print("正在计算 PE_TTM 市值中性化 + 小市值因子...")

    if spot_df.empty:
        print("实时数据为空")
        return pd.DataFrame()

    # 1) 取对数
    df = spot_df[spot_df["pe_ttm"] > 0].copy()
    if df.empty:
        print("没有有效 PE_TTM")
        return pd.DataFrame()

    df["log_pe"] = np.log(df["pe_ttm"])
    df["log_cap"] = np.log(df["market_cap"])

    # 2) 使用统一的 neutralize_factor_sklearn 函数进行中性化
    pe_resid = neutralize_factor_sklearn(
        df["log_pe"].values,
        df["log_cap"].values.reshape(-1, 1)
    )
    df["pe_resid"] = pe_resid

    # 3) 分别标准化：残差 & 市值（负向指标）
    df["pe_resid_z"] = standardize_wide(df["pe_resid"])
    df["cap_z"] = standardize_wide(df["log_cap"])  # 大=正，小=负

    # 4) 等权打分（越小越好）
    df["score"] = 0.5 * (-df["pe_resid_z"]) + 0.5 * (-df["cap_z"])

    # 5) 排序并返回
    result = (df.sort_values("score")
                .head(top_n)
                .reset_index(drop=True))

    return result[["code", "name", "pe_ttm", "market_cap", "score"]]

def get_momentum_pe_factor_stocks(stocks_df, spot_df, top_n=10):
    """
    基于价格动量和估值因子的复合因子选股
    使用统一的 neutralize_factor_sklearn 函数
    """
    print("正在计算基于价格动量和估值因子的复合因子...")
    
    # 获取历史价格数据（过去3个月，约12周）
    start_date = (datetime.now() - timedelta(days=90)).strftime("%Y%m%d")
    end_date = datetime.now().strftime("%Y%m%d")
    
    factor_data = []
    
    for _, row in stocks_df.iterrows():
        code = row["stock_code"]
        name = row["stock_name"]
        
        try:
            # 获取后复权周线数据
            df = ak.stock_zh_a_hist(
                symbol=code,
                period="weekly",
                adjust="hfq",
                start_date=start_date,
                end_date=end_date
            )
            
            if df.empty or len(df) < 13:  # 至少需要13周数据计算12周收益率
                continue
            
            df = df.sort_values("日期").reset_index(drop=True)
            
            # 计算过去12周收益率 (4 * 3)
            current_price = df.iloc[-1]["收盘"]
            price_12_weeks_ago = df.iloc[-13]["收盘"] if len(df) >= 13 else df.iloc[0]["收盘"]
            price_std = (current_price - price_12_weeks_ago) / price_12_weeks_ago
            
            # 从实时数据中获取当前估值和市值数据
            spot_data = spot_df[spot_df["code"] == code]
            if spot_data.empty:
                continue
                
            pb_val = spot_data["pb"].values[0]
            pe_val = spot_data["pe_ttm"].values[0]
            market_cap_val = spot_data["market_cap"].values[0]
            
            # 计算对数市值
            log_mc = np.log(market_cap_val)
            
            factor_data.append({
                "code": code,
                "name": name,
                "price_std": price_std,
                "log_mc": log_mc,
                "pb": pb_val,
                "pe": pe_val,
                "market_cap": market_cap_val
            })
            
        except Exception as e:
            print(f"处理 {code} {name} 时出错: {e}")
            continue
    
    if not factor_data:
        print("没有足够的数据计算复合因子")
        return pd.DataFrame()
    
    factor_df = pd.DataFrame(factor_data)
    
    # 标准化各因子
    factor_df["price_std_std"] = standardize_wide(factor_df["price_std"])
    factor_df["log_mc_std"] = standardize_wide(factor_df["log_mc"])
    factor_df["pb_std"] = standardize_wide(factor_df["pb"])
    factor_df["pe_std"] = standardize_wide(factor_df["pe"])
    
    # 准备控制因子矩阵
    control_factors = np.column_stack([
        factor_df["log_mc_std"].values,
        factor_df["pb_std"].values,
        factor_df["pe_std"].values
    ])
    
    # 使用统一的 neutralize_factor_sklearn 函数进行中性化
    mom_neutralized = neutralize_factor_sklearn(
        factor_df["price_std_std"].values,
        control_factors
    )
    
    factor_df["mom_neutralized"] = mom_neutralized
    factor_df["mom_neutralized_std"] = standardize_wide(factor_df["mom_neutralized"])
    
    # 最终因子得分：取负号（寻找特质性动量弱但估值低的股票）
    factor_df["factor_score"] = -factor_df["mom_neutralized_std"]
    
    # 选择得分最高的top_n只股票
    result = factor_df.sort_values("factor_score", ascending=False).head(top_n).reset_index(drop=True)
    
    return result[["code", "name", "factor_score", "price_std", "pe", "market_cap"]]

def main():
    print("策略开始运行...")
    print("当前时间:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # 1. 获取现金流指数成分股
    stocks_df = get_cs_index_stocks()
    if stocks_df.empty:
        print("成分股获取失败，退出。")
        return

    print(f"共获取成分股 {len(stocks_df)} 只")

    # 2. 一次性获取所有实时数据
    spot_df = get_spot_data_for_stocks(stocks_df)
    if spot_df.empty:
        print("实时数据获取失败，退出。")
        return

    # 3. 从实时数据中获取低PB的10只（底仓，季度调仓）
    low_pb_stocks = get_low_pb_stocks(spot_df, top_n=10)
    print("\n【底仓 - 低PB前10】")
    print(low_pb_stocks[["code", "name", "pb", "market_cap"]])

    # 4. 从实时数据中获取PE_TTM市值中性化最小的10只
    low_pe_neutral_stocks = get_low_pe_market_neutral_stocks(spot_df, top_n=10)
    print("\n【PE_TTM中性化 + 市值因子等权最小前10】")
    print(low_pe_neutral_stocks[["code", "name", "pe_ttm", "market_cap", "score"]])

    # 5. 使用基于价格动量和估值因子的复合因子选股
    momentum_pe_stocks = get_momentum_pe_factor_stocks(stocks_df, spot_df, top_n=10)
    print("\n【机动仓位 - 基于价格动量和估值因子的复合因子前10】")
    print(momentum_pe_stocks[["code", "name", "factor_score", "price_std", "pe"]])


if __name__ == "__main__":
    main()

策略开始运行...
当前时间: 2025-08-28 22:46:41
正在获取指数成分股...
原始列名: ['日期', '指数代码', '指数名称', '指数英文名称', '成分券代码', '成分券名称', '成分券英文名称', '交易所', '交易所英文名称']
成功获取 50 只成分股
共获取成分股 50 只
正在获取所有成分股的实时行情数据...


  0%|          | 0/57 [00:00<?, ?it/s]

获取到 50 只股票的实时数据
过滤PB<=0后剩余: 50 只
过滤PE_TTM<=0后剩余: 49 只
去除缺失值后剩余: 49 只
成功获取 49 只股票的有效实时数据
正在筛选低PB股票...

【底仓 - 低PB前10】
     code  name    pb    market_cap
0  600585  海螺水泥  0.69  1.303099e+11
1  000039  中集集团  0.92  4.427259e+10
2  600219  南山铝业  0.95  4.889355e+10
3  601880  辽港股份  0.98  3.920498e+10
4  600352  浙江龙盛  0.99  3.425758e+10
5  601919  中远海控  1.06  2.469067e+11
6  002120  韵达股份  1.23  2.446921e+10
7  600482  中国动力  1.35  5.287467e+10
8  601877  正泰电器  1.40  5.987028e+10
9  601156  东航物流  1.41  2.474999e+10
正在计算 PE_TTM 市值中性化 + 小市值因子...

【PE_TTM中性化 + 市值因子等权最小前10】
     code   name  pe_ttm    market_cap     score
0  300803    指南针  333.62  9.510865e+10 -2.893322
1  600938   中国海油    8.81  1.224847e+12 -1.373491
2  002352   顺丰控股   21.25  2.439080e+11 -1.124053
3  000858  五 粮 液   12.49  4.870254e+11 -1.099169
4  002714   牧原股份   14.14  2.977210e+11 -0.899614
5  603993   洛阳钼业   14.63  2.537365e+11 -0.829465
6  000792   盐湖股份   22.76  1.041911e+11 -0.654541
7  601225   陕西煤业   12.90  1.970994e+11 -

In [12]:
import akshare as ak
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from sklearn.linear_model import LinearRegression


# 设置显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

def get_cs_index_stocks():
    """
    获取中证指数成分股（如中证红利 000922）
    兼容 akshare 最新列名
    """
    print("正在获取指数成分股...")
    try:
        df = ak.index_stock_cons_csindex(symbol="932368")  # 中证红利指数
        print("原始列名:", df.columns.tolist())

        # 检查必要列是否存在
        if "成分券代码" not in df.columns or "成分券名称" not in df.columns:
            print("错误：缺少必要列 '成分券代码' 或 '成分券名称'")
            return pd.DataFrame(columns=["stock_code", "stock_name"])

        # 提取所需列并重命名
        df = df[["成分券代码", "成分券名称"]].copy()
        df.columns = ["stock_code", "stock_name"]

        # 去除 .SH / .SZ 后缀
        df["stock_code"] = df["stock_code"].str.replace(r"\.SH|\.SZ", "", regex=True)

        print(f"成功获取 {len(df)} 只成分股")
        return df

    except Exception as e:
        print("获取指数成分股失败:", e)
        return pd.DataFrame(columns=["stock_code", "stock_name"])

def get_spot_data_for_stocks(stocks_df):
    """
    一次性获取所有成分股的实时行情数据
    """
    print("正在获取所有成分股的实时行情数据...")
    
    # 获取所有股票的代码列表
    stock_codes = stocks_df["stock_code"].tolist()
    
    try:
        # 一次性获取所有股票的实时数据
        spot_df = ak.stock_zh_a_spot_em()
        
        # 筛选出我们需要的成分股
        spot_df = spot_df[spot_df["代码"].isin(stock_codes)].copy()
        
        if spot_df.empty:
            print("未找到成分股的实时数据")
            return pd.DataFrame()
        
        # 重命名列以便使用
        spot_df = spot_df.rename(columns={
            "代码": "code",
            "名称": "name",
            "市净率": "pb",
            "市盈率-动态": "pe_ttm",
            "总市值": "market_cap"
        })
        
        # 选择需要的列
        spot_df = spot_df[["code", "name", "pb", "pe_ttm", "market_cap"]].copy()
        
        # 记录原始数据量
        original_count = len(spot_df)
        print(f"获取到 {original_count} 只股票的实时数据")
        
        # 过滤异常值：剔除PB<=0、PE_TTM<=0或缺失的值
        spot_df = spot_df[spot_df["pb"] > 0]
        print(f"过滤PB<=0后剩余: {len(spot_df)} 只")
        
        spot_df = spot_df[spot_df["pe_ttm"] > 0]
        print(f"过滤PE_TTM<=0后剩余: {len(spot_df)} 只")
        
        spot_df = spot_df.dropna(subset=["pb", "pe_ttm", "market_cap"])
        print(f"去除缺失值后剩余: {len(spot_df)} 只")
        
        if spot_df.empty:
            print("没有有效的实时数据")
            return pd.DataFrame()
        
        print(f"成功获取 {len(spot_df)} 只股票的有效实时数据")
        return spot_df
        
    except Exception as e:
        print(f"获取实时数据失败: {e}")
        return pd.DataFrame()

def get_low_pb_stocks(spot_df, top_n=10):
    """
    从实时数据中筛选出 PB 最低的 top_n 只股票
    """
    print("正在筛选低PB股票...")
    
    if spot_df.empty:
        print("实时数据为空")
        return pd.DataFrame()
    
    # 确保数据已经过滤了PB<=0的情况
    valid_pb_df = spot_df[spot_df["pb"] > 0].copy()
    
    if valid_pb_df.empty:
        print("没有有效的PB数据")
        return pd.DataFrame()
    
    # 选择PB最小的top_n只股票
    result_df = valid_pb_df.sort_values("pb").head(top_n).reset_index(drop=True)
    
    return result_df[["code", "name", "pb", "market_cap"]]

def get_low_pe_market_neutral_stocks(spot_df, top_n=10):
    """
    方案 A：  
    1. 对 log(PE_TTM) 做市值中性化（回归残差）。  
    2. 对残差与“小市值”分别做 Z-Score 标准化。  
    3. 等权打分：score = 0.5 * (-residual_z) + 0.5 * (-cap_z)  
       负号是因为残差越小、市值越小越好。  
    4. 取分值最小的 top_n 只股票。
    """
    from sklearn.preprocessing import StandardScaler
    print("正在计算 PE_TTM 市值中性化 + 小市值因子...")

    if spot_df.empty:
        print("实时数据为空")
        return pd.DataFrame()

    # 1) 取对数并回归
    df = spot_df[spot_df["pe_ttm"] > 0].copy()
    if df.empty:
        print("没有有效 PE_TTM")
        return pd.DataFrame()

    df["log_pe"] = df["pe_ttm"]
    df["log_cap"] = np.log(df["market_cap"])

    X = df["log_cap"].values.reshape(-1, 1)
    y = df["log_pe"].values
    lr = LinearRegression().fit(X, y)
    df["pe_resid"] = y - lr.predict(X)

    # 2) 分别标准化：残差 & 市值（负向指标）
    scaler = StandardScaler()
    df["pe_resid_z"] = scaler.fit_transform(df[["pe_resid"]])
    df["cap_z"] = scaler.fit_transform(df[["log_cap"]])  # 大=正，小=负

    # 3) 等权打分（越小越好）
    df["score"] = 0.5 * (-df["pe_resid_z"]) + 0.5 * (-df["cap_z"])

    # 4) 排序并返回
    result = (df.sort_values("score")
                .head(top_n)
                .reset_index(drop=True))

    return result[["code", "name", "pe_ttm", "market_cap", "score"]]

def get_low_weekly_return_stocks(stocks_df, top_n=10):
    """
    使用 ak.stock_zh_a_hist(period="weekly", adjust="hfq") 获取后复权周线数据
    开始日期设为 45 天前（确保能获取至少12周数据）
    计算：
      1. 最近一周涨跌幅
      2. 过去12周累计涨跌幅（过滤跌幅 > 30% 的股票）
    返回最近一周涨跌幅最低的 top_n 只股票
    """
    print("正在获取周涨幅数据（后复权，过滤过去12周跌幅>30%）...")

    # 设置开始日期：拉取至少12周数据（约3个月）
    start_date = (datetime.now() - timedelta(days=90)).strftime("%Y%m%d")
    end_date = datetime.now().strftime("%Y%m%d")

    return_list = []

    for _, row in stocks_df.iterrows():
        code = row["stock_code"]
        name = row["stock_name"]
        try:
            # 获取后复权周线数据
            df = ak.stock_zh_a_hist(
                symbol=code,
                period="weekly",
                adjust="hfq",
                start_date=start_date,
                end_date=end_date
            )

            if df.empty or len(df) < 2:
                print(f"{code} {name}: 周线数据不足")
                continue

            # 确保按日期升序排列
            df = df.sort_values("日期").reset_index(drop=True)

            # --- 过滤：过去12周累计跌幅不能超过30% ---
            lookback_weeks = min(12, len(df))  # 最多看12周，如果不足则用全部
            price_12_weeks_ago = df.iloc[-lookback_weeks]["收盘"]
            current_price = df.iloc[-1]["收盘"]
            return_12_weeks = (current_price - price_12_weeks_ago) / price_12_weeks_ago

            if return_12_weeks < -0.3:  # 跌幅 > 30%
                print(f"{code} {name}: 过去{lookback_weeks}周跌幅为 {return_12_weeks:.1%}，已过滤")
                continue

            # --- 计算最近一周涨跌幅 ---
            prev_week = df.iloc[-2]
            curr_week = df.iloc[-1]
            weekly_return = (curr_week["收盘"] - prev_week["收盘"]) / prev_week["收盘"]

            return_list.append({
                "code": code,
                "name": name,
                "weekly_return": weekly_return,
                "12_week_return": return_12_weeks  # 可选：保留长期收益用于分析
            })

        except Exception as e:
            print(f"获取 {code} {name} 数据失败: {e}")
            continue

    # 构建结果 DataFrame
    if not return_list:
        print("没有符合条件的股票。")
        return pd.DataFrame()

    return_df = pd.DataFrame(return_list)
    return_df = return_df.sort_values("weekly_return").head(top_n).reset_index(drop=True)
    return return_df

def main():
    print("策略开始运行...")
    print("当前时间:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # 1. 获取现金流指数成分股
    stocks_df = get_cs_index_stocks()
    if stocks_df.empty:
        print("成分股获取失败，退出。")
        return

    print(f"共获取成分股 {len(stocks_df)} 只")

    # 2. 一次性获取所有实时数据
    spot_df = get_spot_data_for_stocks(stocks_df)
    if spot_df.empty:
        print("实时数据获取失败，退出。")
        return

    # 3. 从实时数据中获取低PB的10只（底仓，季度调仓）
    low_pb_stocks = get_low_pb_stocks(spot_df, top_n=10)
    print("\n【底仓 - 低PB前10】")
    print(low_pb_stocks[["code", "name", "pb", "market_cap"]])

    # 4. 从实时数据中获取PE_TTM市值中性化最小的10只
    low_pe_neutral_stocks = get_low_pe_market_neutral_stocks(spot_df, top_n=10)
    print("\n【PE_TTM中性化 + 市值因子等权最小前10】")
    print(low_pe_neutral_stocks[["code", "name", "pe_ttm", "market_cap", "score"]])

    # 5. 获取周涨幅最低的10只（机动仓位，周调仓）- 这个需要单独获取历史数据
    low_return_stocks = get_low_weekly_return_stocks(stocks_df, top_n=10)
    print("\n【机动仓位 - 周涨幅最低前10】")
    print(low_return_stocks[["code", "name", "weekly_return"]])


if __name__ == "__main__":
    main()

策略开始运行...
当前时间: 2025-08-28 22:03:01
正在获取指数成分股...
原始列名: ['日期', '指数代码', '指数名称', '指数英文名称', '成分券代码', '成分券名称', '成分券英文名称', '交易所', '交易所英文名称']
成功获取 50 只成分股
共获取成分股 50 只
正在获取所有成分股的实时行情数据...


  0%|          | 0/57 [00:00<?, ?it/s]

获取到 50 只股票的实时数据
过滤PB<=0后剩余: 50 只
过滤PE_TTM<=0后剩余: 49 只
去除缺失值后剩余: 49 只
成功获取 49 只股票的有效实时数据
正在筛选低PB股票...

【底仓 - 低PB前10】
     code  name    pb    market_cap
0  600585  海螺水泥  0.69  1.303099e+11
1  000039  中集集团  0.92  4.427259e+10
2  600219  南山铝业  0.95  4.889355e+10
3  601880  辽港股份  0.98  3.920498e+10
4  600352  浙江龙盛  0.99  3.425758e+10
5  601919  中远海控  1.06  2.469067e+11
6  002120  韵达股份  1.23  2.446921e+10
7  600482  中国动力  1.35  5.287467e+10
8  601877  正泰电器  1.40  5.987028e+10
9  601156  东航物流  1.41  2.474999e+10
正在计算 PE_TTM 市值中性化 + 小市值因子...

【PE_TTM中性化 + 市值因子等权最小前10】
     code   name  pe_ttm    market_cap     score
0  300803    指南针  333.62  9.510865e+10 -3.661930
1  600938   中国海油    8.81  1.224847e+12 -1.418346
2  000858  五 粮 液   12.49  4.870254e+11 -0.978498
3  002714   牧原股份   14.14  2.977210e+11 -0.740321
4  002352   顺丰控股   21.25  2.439080e+11 -0.714731
5  603993   洛阳钼业   14.63  2.537365e+11 -0.662452
6  000651   格力电器    9.18  2.646664e+11 -0.624439
7  601919   中远海控    7.04  2.469067e+11 -